# Day 9: Smoke Basin

These caves seem to be [lava tubes](https://en.wikipedia.org/wiki/Lava_tube). Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

```
➁➊➈➈➈➃➂➁➀🄌
➂➈➇➆➇➈➃➈➁➀
➈➇➎➅➆➇➈➇➈➁
➇➆➅➆➇➈➅➆➇➈
➈➇➈➈➈➅➎➅➆➇
```

Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the **low points** - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are **four** low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The **risk level** of a low point is **1 plus its height**. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore **`15`**.

## Part 0

> *I wanted to plot the height map from the input file just for fun. So I did.*
>
> *Unfortunately, it doesn't show up on GitHub, though.*

In [9]:
import plotly.graph_objects as go


with open('./input09.txt') as input_file:
    height_map = [[int(height) for height in line.strip()] for line in input_file]
    input_file.close()

x = list(range(len(height_map[0])))
y = list(range(len(height_map)))

fig = go.Figure(data=[go.Surface(z=height_map, x=x, y=y)])
fig.update_layout(title='Cave Height Map', scene = dict(aspectratio=dict(x=1, y=1, z=0.25), xaxis_title="X", yaxis_title="Y", zaxis_title="Height"), autosize=False, width=700, height=600, margin=dict(l=65, r=50, b=10, t=50))
fig.show()

## Part 1

Find all of the low points on your heightmap. **What is the sum of the risk levels of all low points on your heightmap?**

In [30]:
with open('./input09.txt') as input_file:
    height_map = {(x, y): int(height) for y, line in enumerate(input_file.readlines()) for x, height in enumerate(line.strip())}
    input_file.close()

def get_neighbours(x, y):
    return filter(lambda height: height in height_map, [(x-1, y), (x+1, y), (x, y-1), (x, y+1)])

# find low points and calculate risk level
risk_levels = []
for point in height_map:
    if all(height_map[point] < height_map[neighbour] for neighbour in get_neighbours(*point)):
        risk_levels.append(height_map[point] + 1)

print(f'{len(risk_levels)} Low Points found...\nTotal Risk Level: {sum(risk_levels)}')

236 Low Points found...
Total Risk Level: 575


## Part 2

Next, you need to find the largest basins so you know what areas are most important to avoid.

A **basin** is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The **size** of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

```
➋➊➈➈➈➃➂➁➀🄋
➌➈➇➆➇➈➃➈➁➀
➈➇➄➅➆➇➈➇➈➁
➇➆➅➆➇➈➅➆➇➈
➈➇➈➈➈➅➄➅➆➇
```

The top-right basin, size `9`:

```
➁➀➈➈➈➍➌➋➊🄌
➂➈➇➆➇➈➍➈➋➊
➈➇➄➅➆➇➈➇➈➋
➇➆➅➆➇➈➅➆➇➈
➈➇➈➈➈➅➄➅➆➇
```

The middle basin, size `14`:

```
➁➀➈➈➈➃➂➁➀🄋
➂➈➑➐➑➈➃➈➁➀
➈➑➎➏➐➑➈➇➈➁
➑➐➏➐➑➈➅➆➇➈
➈➑➈➈➈➅➄➅➆➇
```

The bottom-right basin, size `9`:

```
➁➀➈➈➈➃➂➁➀🄋
➂➈➇➆➇➈➃➈➁➀
➈➇➄➅➆➇➈➑➈➁
➇➆➅➆➇➈➏➐➑➈
➈➇➈➈➈➏➎➏➐➑
```

Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = `**`1134`**.

**What do you get if you multiply together the sizes of the three largest basins?**

In [31]:
from math import prod


with open('./input09.txt') as input_file:
    height_map = {(x, y): int(height) for y, line in enumerate(input_file.readlines()) for x, height in enumerate(line.strip())}
    input_file.close()

def get_neighbours(x, y):
    return filter(lambda height: height in height_map, [(x-1, y), (x+1, y), (x, y-1), (x, y+1)])

def get_basin_size(point: tuple[int, int]):
    if height_map[point] == 9:
        return 0
    else:
        del height_map[point]
        return 1 + sum(map(get_basin_size, get_neighbours(*point)))

# find low points
low_points = []
for point in height_map:
    if all(height_map[point] < height_map[neighbour] for neighbour in get_neighbours(*point)):
        low_points.append(point)

# calculate basin sizes
basins = [get_basin_size(point) for point in low_points]

print(f'{len(basins)} Basins found...\nSolution: {prod(sorted(basins, reverse=True)[: 3])}')

236 Basins found...
Solution: 1019700
